In [ ]:
repo_dir = "Repos"  
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-vgg19" / "src" / "vgg19"))
sys.path.insert(0, str(Path.home() / repo_dir / "RETFound_MAE"))
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-exec" / "models" / "vgg19")) 

In [ ]:
# Prerequisites
import json
import os
from eye_ai.eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging

from deriva_ml import DatasetBag, Workflow, ExecutionConfiguration, DatasetVersion
from deriva_ml import MLVocab as vc
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [ ]:
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
catalog_id = "eye-ai" #@param
host = 'www.eye-ai.org'


gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

In [ ]:
cache_dir = '/data'
working_dir = '/data'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

In [ ]:
datasets = [
    '4-4116', # Selected images for training
    '4-411G', # Selected images for testing
    '2-7P5P', # Full multimodal dataset
    ]

to_be_download = []
for dataset in datasets:
    ds_dict = {
        'rid': dataset,
        'materialize':True,
        'version':EA.dataset_version(dataset_rid=dataset),
    }
    to_be_download.append(ds_dict)

workflow_instance = EA.create_workflow(
    name="Multimodal workflow",
    workflow_type="Multimodal workflow"
)

config = ExecutionConfiguration(
    datasets=to_be_download,
    assets = ['2-4JR6',],
    workflow=workflow_instance,
    description="Instance of cropping multimodal images.")

execution = EA.create_execution(config)

In [ ]:
print(execution)

In [ ]:
training_ds_bag = execution.datasets[0]
testing_ds_bag = execution.datasets[1]
multimodal_full_ds_bag = execution.datasets[2]

crop_image_model =  execution.asset_paths["Execution_Asset"][0]

In [ ]:
def get_dataframe_from_bag(ds_bag: DatasetBag, multimodal_full_ds_bag: DatasetBag):
    observation_table = ds_bag.get_table_as_dataframe('Observation')
    image_table = ds_bag.get_table_as_dataframe('Image')
    laterality_table = ds_bag.get_table_as_dataframe('Execution_Image_Fundus_Laterality')

    image_table_filtered = image_table[['RID', 'Filename', 'Observation']].rename(columns={'RID': 'RID_Image'})
    laterality_table_filtered = laterality_table[['Image', 'Image_Side']].rename(columns={'Image': 'RID_Image'})
    image_laterality = pd.merge(image_table_filtered, laterality_table_filtered, left_on='RID_Image', right_on='RID_Image', how='inner')
    observation_table_filtered = observation_table[['RID',  'Subject']].rename(columns={'RID': 'RID_Observation'})
    image_laterality_observation = pd.merge(image_laterality, observation_table_filtered, left_on='Observation', right_on='RID_Observation', how='inner')

    wide = EA.multimodal_wide(multimodal_full_ds_bag) 
    
    image_observation_laterality_subject_wide = pd.merge(
     wide, 
     image_laterality_observation, 
     left_on=['RID_Subject', 'Image_Side'], 
     right_on=['Subject', 'Image_Side'], 
     how='inner'
    )

    return image_observation_laterality_subject_wide

In [ ]:
train_df = get_dataframe_from_bag(training_ds_bag, multimodal_full_ds_bag)
test_df= get_dataframe_from_bag(testing_ds_bag, multimodal_full_ds_bag)

In [ ]:
from pathlib import Path

# Ensure working_dir is a Path object
working_dir = Path("~/working_dir")

# Save DataFrames as CSV files
train_csv_path = working_dir / "train.csv"
test_csv_path = working_dir / "test.csv"

train_df.to_csv(train_csv_path, index=False)
test_df.to_csv(test_csv_path, index=False)

In [ ]:
table_name = 'Image'
EA.find_features(table_name)

In [ ]:
ImageAnnotationFeature = EA.feature_record_class("Image", "Annotation")

In [ ]:
output = execution._working_dir / "image_svg_output"
output.mkdir(parents= True, exist_ok=True)

In [ ]:
from vgg19_disk_crop_predict import preprocess_and_crop
with execution.execute() as exec:
    preprocess_and_crop(
      multimodal_full_ds_bag,
      '~/working_dir/train.csv',
      '~/working_dir/output/output_train.csv',
      'template.jpg',
      str(output),
      crop_image_model,
      "2-NK8E",
      "Optic Nerve",
      False
      )
    
    preprocess_and_crop(
      multimodal_full_ds_bag,
      '~/working_dir/test.csv',
      '~/working_dir/output/output_test.csv',
      'template.jpg',
      str(output),
      crop_image_model,
      "2-NK8E",
      "Optic Nerve",
      False
      )

In [ ]:
ImageAnnotationFeature = EA.feature_record_class("Image", "Annotation")
ImageAnnotationFeature.feature_columns()

In [ ]:
image_rids = []
for file_name in os.listdir(output):
    image_rids.append(file_name.split("_")[1].split('.')[0])

In [ ]:
csv_train = Path("~/working_dir/output/output_train.csv")
df = pd.read_csv(csv_train)
cropping_func_map_train = dict(zip(df["Image RID"], df["Worked Image Cropping Function"]))

In [ ]:
csv_test = Path("~/working_dir/output/output_test.csv")
df = pd.read_csv(csv_test)
cropping_func_map_test = dict(zip(df["Image RID"], df["Worked Image Cropping Function"]))

In [ ]:
# If you don't have the files yet

# for image_rid in image_rids:
#     bounding_box_file = execution.asset_file_path(
#         "Fundus_Bounding_Box", f"Cropped_{image_rid}.svg"
#     )
#     bounding_box_file_paths.append(bounding_box_file)

# target_directory  = os.path.dirname(bounding_box_file_paths[0])
# os.makedirs(target_directory, exist_ok=True)

# for filename in os.listdir(output):
#     source_path = os.path.join(output, filename)
#     if os.path.isfile(source_path):
#         shutil.move(source_path, os.path.join(target_directory, filename))

In [ ]:
import shutil 

bounding_box_file_paths = []
for file_name in os.listdir(output):
    full_path_name = os.path.join(output, file_name)
    bounding_box_file = execution.asset_file_path(
        "Fundus_Bounding_Box", full_path_name, 
    )
    bounding_box_file_paths.append(bounding_box_file)

In [ ]:
bounding_box_file_paths[0]

In [ ]:
image_bounding_box_feature_list = []


for image_rid, asset_file_path in zip(image_rids, bounding_box_file_paths):
    expected_name = f"Cropped_{image_rid}.svg"
    actual_name = os.path.basename(asset_file_path)
    assert expected_name == actual_name, f"Expected {expected_name}, got {actual_name}"

    annotation_func = cropping_func_map_train.get(image_rid) or cropping_func_map_test.get(image_rid)
    if annotation_func:
        image_bounding_box_feature_list.append(
            ImageAnnotationFeature(
                Image=image_rid,
                Execution=execution.execution_rid,
                Fundus_Bounding_Box=asset_file_path,
                Annotation_Function=annotation_func,
                Annotation_Type='Optic Nerve',
            )
        )


In [ ]:
image_bounding_box_feature_list

In [ ]:
with execution.execute() as execution:
    execution.add_features(image_bounding_box_feature_list)

In [ ]:
execution.upload_execution_outputs()